In [ ]:
import os
import numpy as np
import import_ipynb
from tqdm import tqdm as tqdm
import edsr

import matplotlib.image as mpimg
from PIL import Image
import cv2
import tensorflow as tf
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr

In [ ]:
from keras.layers import Conv2D

def compact(x_in):
    conv1 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(x_in)
    conv2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(conv1)
    last = Conv2D(1, strides = 2, kernel_size=(3,3), activation='sigmoid', padding = 'same')(conv2)
    return last

In [ ]:
def compress_this(x_input, quality_factor):
    
    num = x_input.shape[0]
    ans = []
    for i in range(num):
        x_single = x_input[i,:,:,0]
        img = Image.fromarray(np.uint8(x_single*255))
        img.save(os.getcwd()+'/com4.JPEG',"JPEG", quality = quality_factor)
        out_single = mpimg.imread(os.getcwd()+'/com4.JPEG')
        ans.append(out_single)
    ans = np.array(ans)
    ans = np.reshape(ans, (ans.shape[0], ans.shape[1], ans.shape[2], 1))
    return ans

In [ ]:
def ycbcr2rgb(img):
    img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    return img

In [ ]:
def load_images(dset_location):
    loc = dset_location
    images_location = []
    X = []

    print("Extracting image locations..")
    for i in tqdm(os.listdir(loc)):
        images_location.append(loc+'/'+i)

    print("Extracting images..")
    for im_loc in tqdm(images_location):
        img = cv2.imread(im_loc)
        x, y = img.shape[0], img.shape[1]
        if x%2!=0:
            x-=1
        if y%2!=0:
            y-=1
        img = cv2.resize(img, (x, y))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
        X.append(img)
    X = np.array(X)
    return X

In [ ]:
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam


inp = Input(shape=(None, None, 1))
model_edsr = edsr.edsr(x_in=inp, scale=2)
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.00000001, decay=0.0, amsgrad=False)
model_edsr.compile(optimizer=adam, loss='mse')
checkpointr = ModelCheckpoint(filepath='edsr.weights.best.hdf5', save_best_only=True, verbose=1)
model_edsr.summary()


inp = Input(shape=(None, None, 1))
model_comcnn = Model(inp, model_edsr(compact(inp)))
model_comcnn.layers[4].trainable = False
model_comcnn.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
checkpointc = ModelCheckpoint(filepath='comcnn.weights.best.hdf5', save_best_only=True, verbose=1)
model_comcnn.summary()

model_edsr.load_weights('edsr.weights.arb.best.hdf5')
model_comcnn.load_weights('comcnn.weights.arb.best.hdf5')

In [ ]:

def run():
    
    in_loc = 'test_images'
    out_loc = 'output_images'
    val_psnr = []
    val_ssim = []
    
    for im_loc in (os.listdir(in_loc)):
        adr = in_loc + '/' + im_loc

        X = []
        img = cv2.imread(adr)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
        m, n = img.shape[0], img.shape[1]

        flag = 0
        if m%2!=0:
            flag = 1
            m = m - 1

        if n%2!=0:
            flag = 1
            n = n - 1

        img = cv2.resize(img, (m, n))

        X.append(img)
        X = np.array(X)
        X = X.astype('float32')
        X = X / 255

        mpimg.imsave(out_loc+'/'+im_loc+'-in.png', X[0,:,:,0], cmap = 'gray')
        upto_comcnn = Model(model_comcnn.input, model_comcnn.layers[3].output)
        Ycom = upto_comcnn.predict(X[:,:,:,:1])

        Xcompress = compress_this(Ycom, 10)
        Xcompress = Xcompress.astype('float32')
        Xcompress = Xcompress / 255

        Y = model_edsr.predict(Xcompress)
        Y[Y>1] = 1
        mpimg.imsave(out_loc+'/'+im_loc+'-out.png', Y[0,:,:,0], cmap = 'gray')
        val_psnr.append(psnr(Y[0,:,:,0], X[0,:,:,0]))
        val_ssim.append(ssim(Y[0,:,:,0], X[0,:,:,0]))
        print(im_loc + " : ", psnr(Y[0,:,:,0], X[0,:,:,0]))
        print(im_loc + " : ", ssim(Y[0,:,:,0], X[0,:,:,0]))

    print("Average PSNR : ", np.mean(val_psnr))
    print("Average SSIM : ", np.mean(val_ssim))

In [ ]:
# def M_SSIM(y_true , y_pred):
#     ans = tf.image.ssim_multiscale(y_true,y_pred,max_val = 1.0)
#     return ans
# input_img = tf.placeholder(tf.float32, [1,512,768,1])
# input_img1 = tf.placeholder(tf.float32, [1,512,768,1])
# edges_ten = M_SSIM(input_img,input_img1)
# with tf.Session() as sess: msssim = sess.run(edges_ten, feed_dict={input_img:X[0,:,:,0].reshape(1,512,768,1),input_img1:Y[0,:,:,0].reshape(1,512,768,1)})
# print(msssim)